In [39]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
tf.compat.v1.enable_v2_behavior
# Import tensornetwork
import tensornetwork as tn
# Set the backend to tesorflow
# (default is numpy)
tn.set_default_backend("tensorflow")
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data("/hpc/group/carin/fk43/FanjieKong/Megapixels/new_tfquantum/quantum/TensorNetwork/mnist.npz")

In [40]:
from tensorflow.keras.utils import to_categorical

x_train = x_train.reshape((60000, 28, 28, 1)).astype(np.float32)
y_train = to_categorical(y_train, 10).astype(np.float32)
x_test = x_test.reshape((10000, 28, 28, 1))
y_test = to_categorical(y_test, 10)

# print(x_train.shape, y_train.shape)
# print(x_test.shape, y_test.shape)
# print(y_train)

In [41]:
xx_train = (tf.image.resize(x_train, [16,16]).numpy()-128)/255
xx_test = (tf.image.resize(x_test, [16,16]).numpy()-128)/255

AttributeError: 'Tensor' object has no attribute 'numpy'

In [7]:
class TNLayer(tf.keras.layers.Layer):
    
    def __init__(self, dim):
        super(TNLayer, self).__init__()
        # Create the variables for the layer.
        self.dim = dim
        self.a_var = tf.Variable(tf.random.normal(shape=(dim, dim, 2), stddev=1.0/dim), name="a", trainable=True)
        self.b_var = tf.Variable(tf.random.normal(shape=(dim, dim, 2), stddev=1.0/dim), name="b", trainable=True)
        self.bias = tf.Variable(tf.zeros(shape=(dim, dim)), name="bias", trainable=True)

    def call(self, inputs):
        # Define the contraction.
        # We break it out so we can parallelize a batch using tf.vectorized_map.
        def f(input_vec, a_var, b_var, bias_var):
            # Reshape to a matrix instead of a vector.
            input_vec = tf.reshape(input_vec, (self.dim, self.dim))

            # Now we create the network.
            a = tn.Node(a_var)
            b = tn.Node(b_var)
            x_node = tn.Node(input_vec)
            a[1] ^ x_node[0]
            b[1] ^ x_node[1]
            a[2] ^ b[2]

            # The TN should now look like this
            #   |     |
            #   a --- b
            #    \   /
            #      x

            # Now we begin the contraction.
            c = a @ x_node
            result = (c @ b).tensor

            # Finally, add bias.
            return result + bias_var

        # To deal with a batch of items, we can use the tf.vectorized_map function.
        # https://www.tensorflow.org/api_docs/python/tf/vectorized_map
        result = tf.vectorized_map(lambda vec: f(vec, self.a_var, self.b_var, self.bias), inputs)
        return tf.nn.relu(tf.reshape(result, (-1, self.dim**2)))

In [9]:
tn_model = tf.keras.Sequential()
tn_model.add(Conv2D(16, kernel_size=(3,3), activation='relu', name='conv1', input_shape=(28,28,1)))
tn_model.add(Conv2D(32, kernel_size=(3,3), activation='relu', name='conv2'))
tn_model.add(Conv2D(16, kernel_size=(3,3), activation='relu', name='conv3'))
tn_model.add(MaxPooling2D(pool_size=(2,2)))
tn_model.add(tf.keras.layers.Flatten())
tn_model.add(TNLayer(44))
tn_model.add(Dense(10, activation='softmax', name='fc2'))
tn_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 26, 26, 16)        160       
_________________________________________________________________
conv2 (Conv2D)               (None, 24, 24, 32)        4640      
_________________________________________________________________
conv3 (Conv2D)               (None, 22, 22, 16)        4624      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 11, 11, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 1936)              0         
_________________________________________________________________
tn_layer (TNLayer)           (None, 1936)              9680      
_________________________________________________________________
fc2 (Dense)                  (None, 10)               

In [10]:
Dense = tf.keras.layers.Dense
Conv2D = tf.keras.layers.Conv2D
MaxPooling2D = tf.keras.layers.MaxPooling2D

model = tf.keras.Sequential()
model.add(Conv2D(16, kernel_size=(3,3), activation='relu', name='conv1', input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', name='conv2'))
model.add(Conv2D(16, kernel_size=(3,3), activation='relu', name='conv3'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(tf.keras.layers.Flatten())
model.add(Dense(1936, activation='relu', name='fc1'))
model.add(Dense(10, activation='softmax', name='fc2'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 26, 26, 16)        160       
_________________________________________________________________
conv2 (Conv2D)               (None, 24, 24, 32)        4640      
_________________________________________________________________
conv3 (Conv2D)               (None, 22, 22, 16)        4624      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 16)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1936)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 1936)              3750032   
_________________________________________________________________
fc2 (Dense)                  (None, 10)               

In [11]:
class Grid4DMERA(tf.keras.layers.Layer):
    
    def __init__(self, input_dim, bond_dims, output_dims, n_layers=None):
        super(Grid4DMERA, self).__init__()
        # Create the variables for the layer.
        # In this case, the input tensor is (, 1936), we factorize it into a tensor (, 11, 11, 16)
        # first_dim: output shape?
        # second_dim: connect with data tensor
        # third_dim: inter-connect
        if n_layers is None:
            n_layers = np.floor(np.log2(input_dim))
        self.n_layers = n_layers
        in_dims = 16
        dims = input_dim
        self.entanglers = []
        self.isometries= []
            
        # isometries
        self.isometries1 = [tf.Variable(tf.random.normal(shape=(in_dims, in_dims, in_dims, 
                                                                            in_dims, bond_dims)
                                                                     , stddev=1.0/1000),
                                            trainable=True) for j in range(4)]
        
        self.isometries2 = tf.Variable(tf.random.normal(shape=(bond_dims, bond_dims, bond_dims, 
                                                                            bond_dims, output_dims)
                                                                     , stddev=1.0/1000),
                                            trainable=True)

        #print(self.final_mps.shape)
        self.bias = tf.Variable(tf.zeros(shape=(output_dims,)), name="bias", trainable=True)


    def call(self, inputs):
        # Define the contraction.
        # We break it out so we can parallelize a batch using tf.vectorized_map.
        def f(input_vec, isometries1, isometries2, bias_var, n_layers):
            input_vv = []
            #print(input_vec)
            for i in range(4):
                for ii in range(4):
                    input_vv.append(tf.reshape(input_vec[i*4:i*4+4, ii*4:ii*4+4, 0], (1, 16)))
            input_vec = tf.concat(input_vv, axis=0)
            input_vec = tf.reshape(input_vec, (16, 16))
            input_vec = tf.unstack(input_vec)
            input_nodes = []
            for e_iv in input_vec:
                input_nodes.append(tn.Node(e_iv))
            
                
                                     
            isometries_nodes1 = []
            for eiso in isometries1:
                isometries_nodes1.append(tn.Node(eiso))
            isometries_nodes2 = tn.Node(isometries2)
            
             
            
            input_nodes[0][0] ^ isometries_nodes1[0][0]
            input_nodes[1][0] ^ isometries_nodes1[0][1]
            input_nodes[4][0] ^ isometries_nodes1[0][2]
            input_nodes[5][0] ^ isometries_nodes1[0][3]

            input_nodes[2][0] ^ isometries_nodes1[1][0]
            input_nodes[3][0] ^ isometries_nodes1[1][1]
            input_nodes[6][0] ^ isometries_nodes1[1][2]
            input_nodes[7][0] ^ isometries_nodes1[1][3]
            
            input_nodes[8][0] ^ isometries_nodes1[2][0]         
            input_nodes[9][0] ^ isometries_nodes1[2][1]
            input_nodes[12][0] ^ isometries_nodes1[2][2]
            input_nodes[13][0] ^ isometries_nodes1[2][3]
            
            input_nodes[10][0] ^ isometries_nodes1[3][0]
            input_nodes[11][0] ^ isometries_nodes1[3][1]
            input_nodes[14][0] ^ isometries_nodes1[3][2]
            input_nodes[15][0] ^ isometries_nodes1[3][3]
            
            
            isometries_nodes1[0][4] ^ isometries_nodes2[0]
            isometries_nodes1[1][4] ^ isometries_nodes2[1]
            isometries_nodes1[2][4] ^ isometries_nodes2[2]
            isometries_nodes1[3][4] ^ isometries_nodes2[3]
                            
            nodes = tn.reachable(isometries_nodes2)
            result = tn.contractors.greedy(nodes)
            result = result.tensor
            #print(result)
            #result = (c @ b).tensor
            # Finally, add bias.
            return result + bias_var

        # To deal with a batch of items, we can use the tf.vectorized_map function.
        # https://www.tensorflow.org/api_docs/python/tf/vectorized_map
        output = tf.vectorized_map(lambda vec: f(vec, self.isometries1,  self.isometries2, self.bias, self.n_layers), inputs)
        return tf.reshape(output, (-1, 10))

In [12]:
Dense = tf.keras.layers.Dense

MERA_model = tf.keras.Sequential()
MERA_model.add(tf.keras.Input(shape=(16,16,1)))
MERA_model.add(Grid4DMERA(input_dim=8, bond_dims=2, output_dims=10, n_layers=2))
#MERA_model.add(Dense(10, activation='softmax', name='fc'))
MERA_model.add(tf.keras.layers.Softmax())
#MERA_model.build(input_shape=(None, 8, 8, 1))
MERA_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
grid4dmera (Grid4DMERA)      (None, 10)                524458    
_________________________________________________________________
softmax (Softmax)            (None, 10)                0         
Total params: 524,458
Trainable params: 524,458
Non-trainable params: 0
_________________________________________________________________


In [13]:
%%time

# Traditional model
model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
hist = model.fit(x_train, y_train, epochs=20, verbose=1)

Train on 10000 samples
Epoch 1/50
 4736/10000 [=============>................] - ETA: 15s - loss: 2.4036 - accuracy: 0.8235

KeyboardInterrupt: 

In [14]:
%%time

# TensorNetwork model
tn_model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
tn_hist = tn_model.fit(x_train, y_train, epochs=20, verbose=1)

Train on 60000 samples
Epoch 1/50
 5888/60000 [=>............................] - ETA: 1:48 - loss: 0.3984 - accuracy: 0.8740

KeyboardInterrupt: 

In [15]:
%%time

# TensorNetwork model
MERA_model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])
MERA_hist = MERA_model.fit(xx_train, y_train, epochs=20, verbose=1)

ValueError: When using data tensors as input to a model, you should specify the `steps_per_epoch` argument.